In [16]:
filepath_common = 'proturn_output_'
strains = ['aj', 'balbc', 'c57', 'cej', 'dba', 'fvb']
exp = ['ctrl', 'iso'] 
hl = 'hl.out'
dat = 'hl-data_masked'
mask = [1,2,3,4]

In [ ]:
for s in strains: 
    for e in exp: 
        for i in mask: 
            print(f'Rscript mice.R {i} {filepath_common}{s}_{e}/{hl} {filepath_common}{s}_{e}/{dat}_{i}.csv')

In [21]:
import pandas as pd

# Example DataFrames
data_true = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [10, 20, 30],
    'value2': [10, 20, 30]
}
data_pred = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [10, 20, 30],
    'value2': [10, 21, 30]
}

df_true = pd.DataFrame(data_true)
df_pred = pd.DataFrame(data_pred)

In [22]:
import numpy as np 
true = np.array(df_true.drop('ID', axis=1))
pred = np.array(df_pred.drop('ID', axis=1))
true, pred


(array([[10, 10],
        [20, 20],
        [30, 30]]),
 array([[10, 10],
        [20, 21],
        [30, 30]]))

In [23]:
(true - pred)**2

array([[0, 0],
       [0, 1],
       [0, 0]])

In [24]:
#sum of squared differences
np.sum((true - pred)**2)

1

In [27]:
for i in range(10):
    print(i+1)

1
2
3
4
5
6
7
8
9
10


In [12]:
import numpy as np 
import pandas as pd
data_masked = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [np.nan, np.nan, 30],
    'value2': [10, 40, np.nan]
}

data_true = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [10, 3, 30],
    'value2': [10, 20, 30]
}

a = pd.DataFrame(data_masked)
b= pd.DataFrame(data_true)
print(a)
#get all indices where value is null
indices = a.loc[a['value'].isnull(), 'value'].index
print(indices)

print(b.loc[indices])
np.sum(b.loc[indices, 'value'] ** 2)

    ID  value  value2
0  id1    NaN    10.0
1  id2    NaN    40.0
2  id3   30.0     NaN
Index([0, 1], dtype='int64')
    ID  value  value2
0  id1     10      10
1  id2      3      20


109

In [13]:
#keep just 2nd two columns of a
a.iloc[:, 1:]

,value,value2
0,NaN,10.0
1,NaN,40.0
2,30.0,NaN


In [104]:
import pandas as pd 

true = pd.read_csv('proturn_output_aj_ctrl/hl-data_filtered.csv')
true = np.array(true.pivot(index='ID', columns='t', values='A0').reset_index().drop('ID', axis=1))

mean = pd.read_csv('proturn_output_aj_ctrl/si_mean_4.csv')
mean = np.array(mean.drop('Unnamed: 0', axis=1))

knn = pd.read_csv('proturn_output_aj_ctrl/test_4.csv')

knn = np.array(knn.drop(columns=['Unnamed: 0']))

print(knn.shape, mean.shape, true.shape)

er_mean = np.sum((true - mean)**2)
er_dmi = np.sum((true - knn)**2)
print(er_mean, er_dmi)


(8907, 7) (8907, 7) (8907, 7)
784.3082214125543 1319.1539239439942
